# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [40]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [41]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [5]:
# Instantiate the standard scaler method
scaler = StandardScaler() 

# Scale X_train and X_test using StandardScaler
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Ensure X_train and X_test are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_transformed = pd.DataFrame(X_train_transformed, columns=X.columns, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=X.columns, index=X_test.index)

In [8]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
linreg.fit(X_train_transformed, y_train)

# Calculate a baseline r-squared score on training data
baseline_train_r2 = linreg.score(X_train_transformed, y_train)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [48]:
# Your code here

# Set up data structure
interactions = {}

# Find combinations of columns and loop over them
combinations_list = list(combinations(X_train.columns, 2))
for combination in combinations_list:
    # Make copies of X_train and X_test
    X_train_term = X_train[[combination[i] for i in range(len(combination))]]
    X_test_term = X_test[[combination[i] for i in range(len(combination))]]
#     X_train_copy = X_train.copy()
#     X_test_copy = X_test.copy()
    
    # Add interaction term to data

    #X_train_term = X_train_copy[combination]

    # Find r-squared score (fit on training data, evaluate on test data)
    linreg.fit(X_train_term, y_train)
    r_squared = linreg.score(X_test_term, y_test)
    
    # Append to data structure
    interactions[combination] = r_squared
    
    
# Sort and subset the data structure to find the top 7
#interactions
sorted(interactions.items(), key=lambda x:x[1])

[(('LotArea', 'OverallCond'), 0.08952071708905551),
 (('OverallCond', '2ndFlrSF'), 0.12870953711174493),
 (('OverallCond', 'Fireplaces'), 0.17972946226693975),
 (('LotArea', '2ndFlrSF'), 0.18153116815509218),
 (('LotArea', 'Fireplaces'), 0.19627675705108805),
 (('OverallCond', 'TotalBsmtSF'), 0.22591637386074814),
 (('2ndFlrSF', 'Fireplaces'), 0.23616759079320016),
 (('LotArea', 'TotalBsmtSF'), 0.24142139975446486),
 (('TotalBsmtSF', '1stFlrSF'), 0.2567453794393899),
 (('OverallCond', '1stFlrSF'), 0.28479680640550575),
 (('LotArea', '1stFlrSF'), 0.296147638711443),
 (('TotalBsmtSF', 'Fireplaces'), 0.2977083871480546),
 (('1stFlrSF', 'Fireplaces'), 0.3248764279045615),
 (('2ndFlrSF', 'TotRmsAbvGrd'), 0.3314685403596652),
 (('OverallCond', 'TotRmsAbvGrd'), 0.3375422972809694),
 (('LotArea', 'TotRmsAbvGrd'), 0.35687336674071746),
 (('TotRmsAbvGrd', 'Fireplaces'), 0.3717206713455429),
 (('TotalBsmtSF', '2ndFlrSF'), 0.37439231539739504),
 (('OverallCond', 'GarageArea'), 0.37510796939593305)

In [32]:
X_train[['LotArea', 'OverallQual']]

,LotArea,OverallQual
518,9531,6
236,8773,7
38,7922,5
1034,6305,5
520,10800,4
...,...,...
1441,4426,6
1309,7153,6
1237,12393,7
214,10900,6


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_test` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [ ]:
# Your code here

# Loop over top 7 interactions

    # Extract column names from data structure

    # Construct new column name
    
    # Add new column to X_train and X_test


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [ ]:
# Your code here

# Set up data structure

# Loop over all columns

    # Loop over degrees 2, 3, 4
        
        # Make a copy of X_train and X_test
    
        # Instantiate PolynomialFeatures with relevant degree
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
    
        # Find r-squared score
    
        # Append to data structure

# Sort and subset the data structure to find the top 7


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [ ]:
# Your code here

# Filter out duplicates

# Loop over remaining results

    # Create polynomial terms
    
    # Concat new polynomials to X_train and X_test
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [ ]:
# Your code here

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and test data.

In [ ]:
# Your code here

It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and test $R^2$ score and how many features remain.

In [ ]:
# Your code here


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [ ]:
# Your code here


Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

## Evaluate Final Model on Validation Data

### Data Preparation

At the start of this lab, we created `X_val` and `y_val`. Prepare `X_val` the same way that `X_train` and `X_test` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Your code here


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + test set, then find R-Squared and MSE values for the validation set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.